In [1]:
import numpy as np
from scipy.stats import rankdata
from scipy.spatial.distance import pdist, cdist
from enum import Enum
from multiprocessing import Pool, cpu_count

In [2]:
# Distance labels
class Distance(Enum):
    CVM = 0 # ABC-CvM
    MMD = 1 # ABC-MMD
    WASS = 2 # ABC-Wass
    STAT = 3  # ABC-Stat

DISTANCE_LABELS = {Distance.CVM: "CvM", Distance.MMD: "MMD", Distance.WASS: "Wass", Distance.STAT: "Stat"}

In [3]:
# Functions for the distance metrics

# Common to all examples
def cramer_von_mises_distance(obs, sim):
    n = len(obs)
    m = len(sim)
    combined = np.concatenate((obs, sim))
    ranks = rankdata(combined)
    obs_ranks = np.sort(ranks[:n])
    sim_ranks = np.sort(ranks[n:])
    i = np.arange(1, n + 1)
    j = np.arange(1, m + 1)
    term1 = n * np.sum((obs_ranks - i) ** 2)
    term2 = m * np.sum((sim_ranks - j) ** 2)
    denom = n * m * (n + m)
    distance = (term1 + term2) / denom - (4 * n * m - 1) / (6 * (n + m))
    return distance

# Common to all examples
def wasserstein_distance(obs, sim):
    if len(obs) == len(sim):
        sorted_obs = np.sort(obs)
        sorted_sim = np.sort(sim)
        return np.mean(np.abs(sorted_obs - sorted_sim))
    else:
        # fallback to scipy implementation if different sizes
        from scipy.stats import wasserstein_distance as ws
        return ws(obs, sim)

# Common to all examples
def gaussian_kernel(sq_distances, sigma):
    return np.exp(-sq_distances / (2 * sigma))

# Common to all examples
def maximum_mean_discrepancy(obs, sim, obs_sq_dist=None, sigma=None):
    if obs_sq_dist is None:
        obs_sq_dist = pdist(obs.reshape(-1,1), 'sqeuclidean')
    if sigma is None:
        sigma = np.median(obs_sq_dist) ** 0.5
    sim_sq_dist = pdist(sim.reshape(-1,1), 'sqeuclidean')
    mixed_sq_dist = cdist(obs.reshape(-1,1), sim.reshape(-1,1), 'sqeuclidean')
    k_xx = np.mean(gaussian_kernel(obs_sq_dist, sigma))
    k_yy = np.mean(gaussian_kernel(sim_sq_dist, sigma))
    k_xy = np.mean(gaussian_kernel(mixed_sq_dist, sigma))
    return k_xx + k_yy - 2 * k_xy

# Function specific to the exponential family example for ABC-Stat
def stat_distance(observed_data, simulated_data):
    log_observed = np.log(observed_data)
    observed_stats = np.array([
        np.sum(observed_data),
        np.sum(log_observed),
        np.sum(log_observed**2)
    ])
    
    log_simulated = np.log(simulated_data)
    simulated_stats = np.array([
        np.sum(simulated_data),
        np.sum(log_simulated),
        np.sum(log_simulated**2)
    ])
    
    return np.linalg.norm(observed_stats - simulated_stats)

In [4]:
# Function to generate simulated datasets from the three models in the exponential family example. We simule 1/3 of datasets from each mdoel. 
def simulate_datasets(n_sim=10**3, sample_size=100):
    third = n_sim // 3

    # Model 1: Exponential(θ), θ ~ Exponential(1) 
    theta0 = np.random.exponential(scale=1.0, size=third)
    sim0 = np.array([np.random.exponential(scale=1/theta, size=sample_size) for theta in theta0])
    models0 = np.zeros(third, dtype=int)

    # Model 2: LogNormal(θ, 1), θ ~ N(0,1)
    theta1 = np.random.normal(loc=0, scale=1, size=third)
    sim1 = np.array([np.random.lognormal(mean=theta, sigma=1, size=sample_size) for theta in theta1])
    models1 = np.ones(third, dtype=int)

    # Model 3: Gamma(2, θ), θ ~ Exponential(1)
    theta2 = np.random.exponential(scale=1.0, size=third)
    sim2 = np.array([np.random.gamma(shape=2, scale=1/theta, size=sample_size) for theta in theta2])
    models2 = np.full(third, 2, dtype=int)

    # Combine all datasets
    sims = np.vstack((sim0, sim1, sim2))
    thetas = np.concatenate((theta0, theta1, theta2))
    models = np.concatenate((models0, models1, models2))

    return sims, thetas, models

In [5]:
# Compute distances for one observed sample
def compute_distances(observed_sample, sims):
    n_sim = sims.shape[0]
    sample_size = len(observed_sample)

    # Take log of observed data: this is to use in case you want to compute ABC-MMD and ABC-Wass on log-transformed data
    # log_obs = np.log(observed_sample)

    # Precompute squared distances and sigma for MMD
    obs_sq_dist = pdist(observed_sample.reshape(-1,1), 'sqeuclidean')
    sigma = np.median(obs_sq_dist) ** 0.5
    
    distances_cvm = np.zeros(n_sim)
    distances_wass = np.zeros(n_sim)
    distances_mmd = np.zeros(n_sim)
    distances_stat = np.zeros(n_sim)
    
    for i in range(n_sim):
        sim_sample = sims[i]
        distances_cvm[i] = cramer_von_mises_distance(observed_sample, sim_sample)
        distances_wass[i] = wasserstein_distance(observed_sample, sim_sample)
        distances_mmd[i] = maximum_mean_discrepancy(observed_sample, sim_sample, obs_sq_dist, sigma)
        #distances_wass[i] = wasserstein_distance(log_obs, log_sim) # this is to use in case you want to compute ABC-MMD and ABC-Wass on log-transformed data
        #distances_mmd[i] = maximum_mean_discrepancy(log_obs, log_sim, obs_sq_dist, sigma) # this is to use in case you want to compute ABC-MMD and ABC-Wass on log-transformed data
        distances_stat[i] = stat_distance(observed_sample, sim_sample)
    
    return distances_cvm, distances_mmd, distances_wass, distances_stat

In [6]:
# Function to extract results relative to the q% percentile of the distances
def summarize_percentile(thetas, models, distances, percentile):
    n = len(distances)
    k = max(1, round(n * percentile / 100))
    indices = np.argsort(distances)[:k]

    selected_thetas = thetas[indices]
    selected_models = models[indices]

    model_probs = {}
    theta_means = {}

    for m in range(3):  
        mask = selected_models == m
        model_probs[m] = np.mean(mask)  
        if np.any(mask):
            theta_means[m] = np.mean(selected_thetas[mask])  
        else:
            theta_means[m] = np.nan  

    return model_probs, theta_means

In [7]:
# Run ABC for one observed dataset
def run_abc_for_one_observed(args):
    observed_sample, sims, thetas, models, percentiles = args

    # Compute distances
    dist_cvm, dist_mmd, dist_wass, dist_stat = compute_distances(observed_sample, sims)

    results = {}
    for dist_enum, dist_array in zip(Distance, [dist_cvm, dist_mmd, dist_wass, dist_stat]):
        dist_name = DISTANCE_LABELS[dist_enum]
        results[dist_name] = {
            'model_probs': [],  
            'theta_means': []   
        }

        for perc in percentiles:
            model_probs, theta_means = summarize_percentile(thetas, models, dist_array, perc)
            results[dist_name]['model_probs'].append(model_probs)
            results[dist_name]['theta_means'].append(theta_means)

    return results

In [ ]:
# Main function
def main():
    np.random.seed(42)
    sample_size = 100
    n_sim = 10**6
    n_observed = 100
    percentiles = [0.1, 0.05, 0.01]  # Change for the percentiles you want

    print("Simulating datasets...")
    sims, thetas, models = simulate_datasets(n_sim=n_sim, sample_size=sample_size)

    print(f"Simulating {n_observed} observed datasets...")
    observed_datasets = [np.random.exponential(scale=2.0, size=sample_size) for _ in range(n_observed)] # This is simulation from model M1 (Exponential(theta), you can change for other models

    # Prepare arguments for parallel ABC
    args_list = [(obs, sims, thetas, models, percentiles) for obs in observed_datasets]

    print(f"Starting parallel ABC with {cpu_count()} cores...")
    with Pool() as pool:
        all_results = pool.map(run_abc_for_one_observed, args_list)

    # Aggregate results
    distance_names = list(DISTANCE_LABELS.values())
    n_dist = len(distance_names)
    n_perc = len(percentiles)
    n_models = 3

    model_probs_summary = np.zeros((n_observed, n_dist, n_perc, n_models))
    theta_means_summary = np.zeros((n_observed, n_dist, n_perc, n_models))

    for i, res in enumerate(all_results):
        for d_idx, dist_name in enumerate(distance_names):
            for p_idx in range(n_perc):
                model_probs = res[dist_name]['model_probs'][p_idx]
                theta_means = res[dist_name]['theta_means'][p_idx]
                for m in range(n_models):
                    model_probs_summary[i, d_idx, p_idx, m] = model_probs.get(m, 0.0)
                    theta_means_summary[i, d_idx, p_idx, m] = theta_means.get(m, np.nan)

    # Save results
    np.savez('expo_family_ex_expo.npz',
             model_probs=model_probs_summary,
             theta_means=theta_means_summary,
             percentiles=percentiles,
             distance_names=distance_names,
             model_labels=["Expo", "LogN", "Gamma"])

    print("ABC summaries saved to expo_family_ex_expo.npz")

if __name__ == "__main__":
    main()